In [128]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [90]:
import glob

In [91]:
source_csv = glob.glob('../lvrland_crawler/lvr_src/*/*/*/*/*_lvr_land_*.csv')
print(len(source_csv))

108


In [102]:
h0 = []
h1 = []
for s in source_csv:
    sp = s.split('/')
    year, season, trade_type, city = sp[3], sp[4], sp[5], sp[6]
    new_column = f'{year}_{season}_{city}_{trade_type}'
    temp_df_h0 = pd.read_csv(s)
    temp_df_h1 = pd.read_csv(s, header=1)

    temp_df_h0['df_name'] = new_column
    temp_df_h1['df_name'] = new_column
    
    h0.append(temp_df_h0)
    h1.append(temp_df_h1)

In [103]:
# dataframe
df_source = pd.concat(h0, axis=0,ignore_index=True)
df_lvrland = pd.concat(h1, axis=0,ignore_index=True)

In [104]:
# 主要用途為住家用
is_self_residence = df_lvrland['main use']=='住家用'
is_self_residence

0          True
1         False
2          True
3          True
4          True
          ...  
626536     True
626537    False
626538    False
626539    False
626540    False
Name: main use, Length: 626541, dtype: bool

In [105]:
# 建物型態為住宅大樓
is_self_residence_building = df_lvrland['building state'].fillna('0').str.startswith('住宅大樓')
is_self_residence_building


0         False
1         False
2         False
3         False
4         False
          ...  
626536     True
626537    False
626538    False
626539    False
626540    False
Name: building state, Length: 626541, dtype: bool

In [106]:
import cn2an

In [107]:
# 樓層資料包含有中文數字+層數, 數字, 地下層，將其轉為integer
def convert_floor2int(x):
    if type(x) == str:
        if x.isdigit():
            return int(x)
        else:
            if x != '地下層':
                x = cn2an.cn2an(x.strip("層"))
                return x
    elif type(x) == int:
        return x
    else:
        print(x)
    return 0
        

In [109]:
# 總樓層數大於等於十三層
is_gte_floor_13 = df_lvrland['total floor number'].fillna(0).map(convert_floor2int) >= 13
is_gte_floor_13

0         False
1         False
2         False
3         False
4         False
          ...  
626536     True
626537     True
626538     True
626539     True
626540     True
Name: total floor number, Length: 626541, dtype: bool

In [126]:
# 3 filter dataframe
df_residence_gte_13 = df_lvrland[(is_self_residence & is_self_residence_building & is_gte_floor_13)]
df_residence_gte_13.shape

(129870, 29)

In [131]:
# 增加年度season欄位
df_residence_gte_13['year_season'] = df_residence_gte_13['df_name'].map(lambda x: '_'.join([x.split('_')[0],x.split('_')[1]]))
df_residence_gte_13['year_season']

14        104_1
17        104_1
18        104_1
21        104_1
28        104_1
          ...  
626532    108_2
626533    108_2
626534    108_2
626535    108_2
626536    108_2
Name: year_season, Length: 129870, dtype: object

In [142]:
# 增加縣市別
city_map = {
    'A': '台北市',
    'B': '台中市',
    'F': '新北市',
    'H': '桃園市',
    'E': '高雄市'
    
    
}
df_residence_gte_13['city'] = df_residence_gte_13['df_name'].map(lambda x: city_map[x.split('_')[2]])
df_residence_gte_13['city']

14        台北市
17        台北市
18        台北市
21        台北市
28        台北市
         ... 
626532    台中市
626533    台中市
626534    台中市
626535    台中市
626536    台中市
Name: city, Length: 129870, dtype: object

In [351]:
import datetime
def data_interval(data1,data2):
    d1 = datetime.datetime.strptime(data1, '%Y%m%d')
    d2 = datetime.datetime.strptime(data2, '%Y%m%d')
    delta = d1 - d2
    year = int(delta.days / (365.25))
    if year < 0:
        year = 0
    return year

In [352]:
def convert_tw_year(data):
    tw_y = str(int(data))[:-4]
#     print(tw_y)
    y = str(int(tw_y) + 1911)
    y_convert = y + str(str(int(data))[-4:])
    return y_convert

In [353]:
def get_interval(x):
    try:
        d1 = x['transaction year month and day']
        d2 = x['construction to complete the years']
        d1_convert = convert_tw_year(d1)
        d2_convert = convert_tw_year(d2)
    #     print(d1_convert, d2_convert)
        return data_interval(d1_convert, d2_convert)
    except:
        return 0

In [354]:
# 建物年齡（交易日期-建築完成年月）
df_residence_gte_13['building_age'] = df_residence_gte_13.fillna(0).apply(get_interval, axis = 1)

In [355]:
df_residence_gte_13

,The villages and towns urban district,transaction sign,land sector position building sector house number plate,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,transaction pen number,shifting level,...,the unit price (NTD / square meter),the berth category,berth shifting total area square meter,the berth total price NTD,the note,serial number,df_name,year_season,city,building_age
14,中山區,房地(土地+建物),臺北市中山區長安東路二段181~210號,12.88,商,NaN,NaN,1030927,土地1建物1車位0,八層,...,285526.0,NaN,0.00,0,含增建或未登記建物。,RPQOMLNJJIJFFCA96CA,104_1_A_A,104_1,台北市,7
17,中山區,房地(土地+建物)+車位,臺北市中山區林森北路571~600號,9.37,商,NaN,NaN,1031003,土地1建物1車位1,八層,...,195141.0,坡道平面,39.18,0,NaN,RPRNMLOJJIJFFCA07CA,104_1_A_A,104_1,台北市,3
18,內湖區,房地(土地+建物)+車位,臺北市內湖區行善路241~270號,33.44,其他,NaN,NaN,1030920,土地1建物1車位2,十五層,...,190150.0,坡道平面,49.78,0,NaN,RPTNMLOJJIJFFCA27CA,104_1_A_A,104_1,台北市,4
21,內湖區,房地(土地+建物),臺北市內湖區民權東路六段296巷1~30號,20.07,住,NaN,NaN,1031020,土地1建物1車位0,十六層，電梯樓梯間,...,256840.0,NaN,0.00,0,NaN,RPXUMLPJJIJFFCA48CA,104_1_A_A,104_1,台北市,18
28,中山區,房地(土地+建物)+車位,臺北市中山區長春路91~120號,29.36,商,NaN,NaN,1030825,土地1建物1車位1,四層,...,256993.0,坡道平面,46.12,2500000,NaN,RPOQMLTKJIJFFCA67CA,104_1_A_A,104_1,台北市,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626532,梧棲區,房地(土地+建物)+車位,梧棲段2821~2850地號,0.06,商,NaN,NaN,1070907,土地1建物1車位1,十一層,...,55574.0,坡道平面,31.04,520000,NaN,RPSOMLTJNHOFFFB09CB,108_2_B_B,108_2,台中市,0
626533,梧棲區,房地(土地+建物)+車位,梧棲段2821~2850地號,0.06,商,NaN,NaN,1070629,土地1建物1車位1,十三層,...,55904.0,坡道平面,31.04,580000,NaN,RPTOMLTJNHOFFFB56CB,108_2_B_B,108_2,台中市,0
626534,梧棲區,房地(土地+建物)+車位,梧棲段2821~2850地號,0.06,商,NaN,NaN,1070629,土地1建物1車位1,十五層,...,55409.0,坡道平面,31.04,580000,NaN,RPVOMLTJNHOFFFB76CB,108_2_B_B,108_2,台中市,0
626535,梧棲區,房地(土地+建物)+車位,梧棲段2821~2850地號,0.16,商,NaN,NaN,1070930,土地1建物1車位1,九層,...,48336.0,坡道平面,29.95,670000,NaN,RPWOMLTJNHOFFFB86CB,108_2_B_B,108_2,台中市,0


In [356]:
# filter.csv

# import numpy as np
# output = np.vstack([df_residence_gte_13.columns.values, df_residence_gte_13.values])
# pd.DataFrame(output, columns=df_source.columns).to_csv('filter.csv', index=0)
pd.DataFrame(df_residence_gte_13).to_csv('filter.csv', index=0)

In [62]:
# 總件數
total_case = df_residence_gte_13.shape[0]
total_case

129870

In [67]:
# 總車位數(透過交易筆棟數)
total_berth = df_residence_gte_13['transaction pen number'].map(lambda x: int(x.split('車位')[-1])).sum()
total_berth

110401

In [68]:
# 平均總價元
avg_total_price = df_residence_gte_13['total price NTD'].mean(0).round()
avg_total_price

16080873.0

In [69]:
# 平均車位總價元(有車位才算，價格大於0)
avg_total_berth_price = df_residence_gte_13[df_residence_gte_13['the berth total price NTD']> 0]['the berth total price NTD'].mean().round()
avg_total_berth_price

1992633.0

In [66]:
# count.csv
data = {
    '總件數': [total_case],
    '總車位數': [total_berth],
    '平均總價元': [avg_total_price],
    '平均車位總價元': [avg_total_berth_price]
}

pd.DataFrame(data, columns=['總件數','總車位數','平均總價元', '平均車位總價元']).to_csv('count.csv', index=0)

In [285]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(
    ['localhost'],
    port=9200

)

In [288]:
mapping = {
  "settings": {
    "number_of_shards": 1
  },
  "mappings": {
    "properties": {
      "Building present situation pattern - room": {
        "type": "long"
      },
      "The villages and towns urban district": {
        "type": "keyword"
      },
      "Whether there is manages the organization": {
        "type": "keyword"
      },
      "berth shifting total area square meter": {
        "type": "double"
      },
      "building present situation pattern - compartmented": {
        "type": "keyword"
      },
      "building present situation pattern - hall": {
        "type": "long"
      },
      "building present situation pattern - health": {
        "type": "long"
      },
      "building shifting total area": {
        "type": "double"
      },
      "building state": {
        "type": "keyword"
      },
      "construction to complete the years": {
        "type": "double"
      },
      "df_name": {
        "type": "keyword"
      },
      "land sector position building sector house number plate": {
        "type": "keyword"
      },
      "land shifting total area square meter": {
        "type": "double"
      },
      "main building materials": {
        "type": "keyword"
      },
      "main use": {
        "type": "keyword"
      },
      "serial number": {
        "type": "keyword"
      },
      "shifting level": {
        "type": "keyword"
      },
      "the berth category": {
        "type": "keyword"
      },
      "the berth total price NTD": {
        "type": "long"
      },
      "the note": {
        "type": "text"
      },
      "the unit price (NTD / square meter)": {
        "type": "double"
      },
      "the use zoning or compiles and checks": {
        "type": "keyword"
      },
      "total floor number": {
        "type": "keyword"
      },
      "total price NTD": {
        "type": "long"
      },
      "transaction pen number": {
        "type": "keyword"
      },
      "transaction sign": {
        "type": "keyword"
      },
      "transaction year month and day": {
        "type": "long"
      }
    }
  }
}

In [ ]:
# create index
response = es.indices.create(
    index="csv_data",
    body=mapping,
    ignore=400 # ignore 400 already exists code
)


In [ ]:
# 傳送至 Elasticsearch
import csv
with open('filter.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='csv_data')